In [1]:
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

In [2]:
df = pd.read_excel("data.xlsx", sheet_name=None, header=0)

In [3]:
all_df = {}
for i, d in enumerate(df.items()):
    name, data = d
    all_df[name] = data

In [4]:
data = all_df['1K-01']

In [5]:
data = data[data['DEPTH'] == data["DEPTH"]]   # drop NaN value row

In [6]:
data = data.drop(["Core_Facies"], axis=1)

In [8]:
data['Log_Facies'].apply(lambda x: str(x)).describe()

count     1025
unique       3
top        1.0
freq       530
Name: Log_Facies, dtype: object

In [9]:
comb = [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
def label_facies(row):
    f = row['Fluid']
    lf = row['Log_Facies']
    return comb.index((f, lf))
data['Color_b'] = data.apply(label_facies, axis=1)

## L R

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [11]:
X = data[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]
ss = StandardScaler()
X = ss.fit_transform(X)

In [12]:
Y = data['Color_b']

In [13]:
clf = LogisticRegression()

In [14]:
clf.fit(X, Y)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
Y_pred = clf.predict(X)

In [16]:
from sklearn.metrics import confusion_matrix 

In [17]:
results = confusion_matrix(Y, Y_pred)
results

array([[ 91,  12,   1,   0,   0],
       [ 21, 120,   8,   4,   0],
       [  1,  11,  52,   0,   0],
       [  5,   0,   0, 377,  44],
       [  1,  11,   0,  77, 189]])

In [18]:
from sklearn.metrics import classification_report
report = classification_report(Y, Y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.88      0.82       104
           1       0.78      0.78      0.78       153
           2       0.85      0.81      0.83        64
           3       0.82      0.88      0.85       426
           4       0.81      0.68      0.74       278

   micro avg       0.81      0.81      0.81      1025
   macro avg       0.81      0.81      0.80      1025
weighted avg       0.81      0.81      0.81      1025



In [20]:
print(clf.intercept_)
print(clf.coef_)

[-5.08005733 -4.311037   -5.331887   -1.14080956 -1.81354935]
[[-2.43188111 -0.90211191 -0.13653637 -0.91506447  2.73632879 -2.01120305
   0.01514691 -0.42972747 -1.56039477]
 [-0.40884009 -3.40720428  0.90379047  0.6063032  -0.34929221  0.91425576
  -0.77280182  0.09841814  1.34329582]
 [-3.2547469   1.09666476  0.09938329  0.08917029 -1.33224463  0.83745061
   0.0253272   0.24571485  1.03975919]
 [-0.25275672 -0.20518888 -0.73024109  0.61602753 -3.27707798  0.43851904
   0.89008507 -1.16838217 -2.75825879]
 [ 1.08032958  0.30409153  0.1445996  -0.24896571 -0.71530434  0.14849933
  -0.09512613  0.30873129  1.6432822 ]]


In [21]:
data_test = all_df['1K-01']
data_test = data_test[data_test['DEPTH'] == data_test["DEPTH"]]  
data_test = data_test.drop(["Core_Facies"], axis=1)

data_test['Color_b'] = data_test.apply(label_facies, axis=1)
X_test = data_test[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]
X_test = ss.fit_transform(X_test)
Y_test = data['Color_b']
Y_test_pred = clf.predict(X_test)

In [22]:
results = confusion_matrix(Y_test, Y_test_pred)
results

array([[ 91,  12,   1,   0,   0],
       [ 21, 120,   8,   4,   0],
       [  1,  11,  52,   0,   0],
       [  5,   0,   0, 377,  44],
       [  1,  11,   0,  77, 189]])

In [23]:
report = classification_report(Y_test, Y_test_pred)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.88      0.82       104
           1       0.78      0.78      0.78       153
           2       0.85      0.81      0.83        64
           3       0.82      0.88      0.85       426
           4       0.81      0.68      0.74       278

   micro avg       0.81      0.81      0.81      1025
   macro avg       0.81      0.81      0.80      1025
weighted avg       0.81      0.81      0.81      1025



In [25]:
Y_t = Y_test[Y_test_pred==Y_test]

In [27]:
Y_t.shape[0]/Y_test.shape[0]

0.8087804878048781